In [3]:
!pip install PyMuPDF docx2txt pytesseract pillow pix2tex

Defaulting to user installation because normal site-packages is not writeable


In [19]:
import fitz
import docx2txt
import os
import pytesseract
from PIL import Image
from pix2tex.cli import LatexOCR
from IPython.display import display, Latex, Math

# PDF Text and Image Extraction

In [5]:
doc = fitz.open(r"Report.pdf")

for page_num, page in enumerate(doc):
    # Extract text
    text = page.get_text()
    print(f"Text on Page {page_num + 1}:\n{text}\n")
    
    # Extract images
    image_list = page.get_images(full=True)
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_filename = f"page_{page_num + 1}_image_{image_index}.png"
        with open(image_filename, "wb") as img_file:
            img_file.write(image_bytes)
        print(f"Saved Image: {image_filename}")

doc.close()  


Text on Page 1:
SCIENCE LAB REPORT
Lorem ipsum dolor sit amet, consectetuer adipiscing elit
Your Name
09.04.20XX
8TH GRADE SCIENCE


Saved Image: page_1_image_1.png
Text on Page 2:
INTRODUCTION
Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh
euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim
veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea
commodo consequat.
HYPOTHESIS
Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat,
vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.
MATERIALS
1. Lorem ipsum dolor sit amet
2. Consectetuer adipiscing elit
3. Sed diam nonummy nibh euismod
PROCEDURE
1. Lorem ipsum dolor sit amet
2. Consectetuer adipiscing elit
3. Sed diam nonummy nibh euismod
DATA
LOREM IPSUM
DOLOR SIT
Lorem ipsum
Lorem ipsum
Lorem ipsum
1


Text on Page 3:
RESULTS
Nam liber tempor cum soluta nobis eleifend option congue nihil

# Word Text and Image Extraction

In [6]:
# Specify the path to your Word document
doc_path = r"Report.docx"
# Specify the directory to save images
images_dir = r"images_directory"  # Adjust the path as needed

# Ensure the images directory exists
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Extract text and save images from the document
text = docx2txt.process(doc_path, images_dir)

# Print the extracted text
print(text)

SCIENCE LAB REPORT

Lorem ipsum dolor sit amet, consectetuer adipiscing elit



Your Name

09.04.20XX

8TH GRADE SCIENCE









INTRODUCTION

Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat.

HYPOTHESIS

Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.

MATERIALS

Lorem ipsum dolor sit amet

Consectetuer adipiscing elit

Sed diam nonummy nibh euismod

PROCEDURE

Lorem ipsum dolor sit amet

Consectetuer adipiscing elit

Sed diam nonummy nibh euismod

DATA

LOREM IPSUM

DOLOR SIT



Lorem ipsum





Lorem ipsum





Lorem ipsum





RESULTS

Nam liber tempor cum soluta nobis eleifend option congue nihil imperdiet doming id quod mazim placerat facer possim as

# OCR Math Equation Extraction

In [20]:
img = Image.open(r"C:\Users\arnav\Downloads\Screenshot 2024-02-17 085808.png")
model = LatexOCR()

# Your LaTeX equation as a Python string
latex_str = model(img)

# Display the LaTeX equation
display(Math(latex_str))

<IPython.core.display.Math object>